In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from skmultilearn.adapt import MLkNN
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    hamming_loss as hl,)

from sklearn.metrics import (
    label_ranking_loss,
    coverage_error,
    average_precision_score,)

import multilabel_knn as mlk

In [26]:
data = pd.read_csv('/home/adel/Documents/Code/Ant-Miner/datasets/multi_label/cleaned/emotions.csv')
labels = [col for col in data.columns if 'label' in col]
X = data.drop(columns=labels)
y = data[labels]
y = y.values.astype(int)


In [27]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Initialize the MLkNN classifier
model = mlk.multilabel_kNN(k=10)
model.fit(X_train, y_train)



In [28]:
y_pred, likelihood = model.predict(X_test)


In [29]:
# Accuracy 
def accuracy(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    accs = []
    for i in range(y_true.shape[1]):
        accs.append(accuracy_score(y_true[:, i], y_pred[:, i]))
    return np.mean(accs)

In [30]:
y_pred_dense = y_pred.toarray().astype(int)

y_proba_dense = likelihood.toarray().astype(float)

In [31]:
acc = accuracy(y_test, y_pred_dense)
sub_acc = accuracy_score(y_test, y_pred_dense)
f1_micro = f1_score(y_test, y_pred_dense, average='micro')
f1_macro = f1_score(y_test, y_pred_dense, average='macro')
hm = hl(y_test, y_pred_dense)
rl = label_ranking_loss(y_test, y_proba_dense)
ce = coverage_error(y_test, y_proba_dense)
ap = average_precision_score(y_test, y_proba_dense, average='weighted')

print(f'Accuracy: {acc*100:.4f} - Subset Accuracy: {sub_acc*100:.4f} - F1 Score (Micro): {f1_micro*100:.4f} - F1 Score (Macro): {f1_macro*100:.4f} - Hamming Loss: {hm:.4f} - Ranking Loss: {rl:.4f} - Coverage Error: {ce:.4f} - Average Precision: {ap*100:.4f}')

Accuracy: 76.8727 - Subset Accuracy: 22.4719 - F1 Score (Micro): 66.4858 - F1 Score (Macro): 64.0434 - Hamming Loss: 0.2313 - Ranking Loss: 0.2606 - Coverage Error: 3.4101 - Average Precision: 42.0457
